In [1]:
import numpy as np
import casadi as ca
import scipy.integrate
import matplotlib.pyplot as plt

In [ ]:
def euler_kinematics(e, w):
    '''Derivative of Euler angles'''
    v = ca.SX.sym('v',3)
    v[0] = (w[1]*ca.sin(e[2])+w[2]*ca.cos(e[2]))/ca.cos(e[1])
    v[1] = w[1]*ca.cos(e[2]) - w[2]*ca.sin(e[2])
    v[2] = w[0] + (w[1]*ca.sin(e[2]+w[2]*ca.cos(e[2])))*ca.tan(e[1])
    return v

def euler_to_dcm(e):
    ''' Transition from Euler angles to direction cosine matrix'''
    R = ca.SX.sym('R', 3)
    
    return R

In [ ]:
def derive_equations():
    arm_angles_deg = [45, -135, -45, 135]
    motor_dirs = [1, 1, -1, -1]

    mix_matrix = ca.DM([
        # roll, pitch, yaw, thrust
        [   -1,     1,   1,     1], # duty motor 0
        [    1,    -1,   1,     1], # duty motor 1
        [    1,     1,  -1,     1], # duty motor 2
        [   -1,    -1,  -1,     1], # duty motor 3
    ])

    # state (x)
    x = ca.SX.sym('x',12)
    omega_b = x[0:3]                       # Angular velocity (body)
    vel_b = x[3:6]                         # Velocity (body)
    pos_n = x[6:9]                         # Position (inertial)
    euler = x[9:12]                        # Orientation (inertial) = r_nb

    x0 = [0]*12

    # input
    n_motors = len(arm_angles_deg)
    u = ca.SX.sym('u', 4)
    duty_cmd = ca.mtimes(mix_matrix, u)

    # parameters
    l_arm = ca.SX.sym('l_arm')
    m = ca.SX.sym('m')
    rho = ca.SX.sym('rho')
    Jx = ca.SX.sym('Jx')
    Jy = ca.SX.sym('Jy')
    Jz = ca.SX.sym('Jz')
    CT = ca.SX.sym('CT')
    CM = ca.SX.sym('CM')
    kV = ca.SX.sym('kV')
    vBatt = ca.SX.sym('vBatt')
    r_prop = ca.SX.sym('r_prop')
    g = ca.SX.sym('g')
    tau = ca.SX.sym('tau')
    p = ca.vertcat(l_arm, m, kV, vBatt, rho, Jx, Jy, Jz, CT, CM, r_prop, g, tau)
    p0 = [1, 1, 1550, 11.1, 1.225, 1, 1, 1, 2.7e-4, 5e-5, 0.1778/2, 9.8, 0.015]
    # l_arm: length of rotor arm [m]
    # m: mass of vehicle [kg]
    # kV: kV: motor kV constant, [rpm/V]
    # vBatt: voltage of battery [V]
    # rho: density of air [kg/m^3]
    # Jx: moment of inertia to xx
    # Jy: moment of inertia to yy
    # Jz: moment of inertia to zz
    # CT: Thrust coeff
    # CM: Moment coeff
    # r_prop: radius of propeller
    # g: gravity constant [m/s^2]
    # tau: 

    J = ca.diag(ca.vertcat(Jx, Jy, Jz))                                             # Moment of inertia of quadrotor

    # motor model
    def motor_model(duty, motor_dir):
        omega = duty*vBatt*kV*(2*ca.pi/60)                                          # duty * rad/s
        V = r_prop*duty*vBatt*kV                                                    # round * m/min
        s = ca.pi*r_prop**2                                                         # Area of rotor
        q = rho*V**2/2
        return ca.vertcat(0, 0, -CT*q*s), ca.vertcat(0, 0, motor_dir*CM*q*s)        # rotor thrust(F) and Moment

    # forces and moments
    C_nb = euler_to_dcm(euler)                                                # from euler to direction cosine matrix
    F_b = ca.mtimes(C_nb, ca.vertcat(0, 0, m*g))                                  # Body force Initialize
    M_b = ca.SX.zeros(3)                                                            # Body moment(torque) Initialize

    for i in range(n_motors):
        arm_angle = arm_angles_deg[i]*ca.pi/180                                     # deg -> rad
        ri_b = ca.vertcat(l_arm*ca.cos(arm_angle), l_arm*ca.sin(arm_angle), 0)      # vector to each motor from center
        Fi_b, Mi_b = motor_model(duty_cmd[i], motor_dirs[i])                        # get each rotor's F and M
        F_b += Fi_b                                                                 # sum up all rotor F vector
        M_b += Mi_b + ca.cross(ri_b, Fi_b)                                          # sum up all rotor M vector & M from each leg

    x_dot = ca.vertcat(
            ca.mtimes(ca.inv(J), M_b - ca.cross(omega_b, ca.mtimes(J, omega_b))),    # omega dot (angular acceleration)
            F_b/m - ca.cross(omega_b, vel_b),                                        # v dot (acceleration)
            euler_kinematics(euler, omega_b),                                        # omega (angular velocity) (inertial)
            ca.mtimes(C_nb, vel_b),                                                  # v (velocity) (inertial)
        )

In [ ]:
def simulate(eqs, dt, tf):
    eqs = derive_equations()
    x0 = eqs['x0']
    p0 = eqs['p0']
    t_vect = np.arange(0, tf, dt)
    xi = x0
    data = {
        't': [],
        'x': [],
        'u': [],
        'euler': [],
    }
    for t in t_vect:
        u = np.array(eqs['control'](xi, p0)).reshape(-1)

        data['x'].append(xi)
        data['u'].append(u)
        data['t'].append(t)
        data['euler'].append(np.array(eqs['mrp_to_euler'](xi[6:10])).reshape(-1))
        
        res = scipy.integrate.solve_ivp(fun=lambda t, x: np.array(eqs['rhs'](xi, u, p0)).reshape(-1), t_span=[t, t+dt], t_eval=[t+dt], y0=xi)
        xi = res['y']
        xi[6:10] = eqs['shadow_if_necessary'](xi[6:10])
        xi = eqs['saturate'](xi)
        xi = np.array(xi).reshape(-1)

    for k in data.keys():
        data[k] = np.array(data[k])
    return data